In [1]:
pip install faiss-cpu sentence-transformers pandas tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 65.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 91.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.5/561.5 kB 62.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 93.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 99.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.4/35.4 MB 3.0 MB/s eta 0:00:0000:0100:01mm
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 389.4 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.9/798.9 kB 293.6 kB/s eta 0:00:00a 0:00:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 305.4 kB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install biopython

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install datasets

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 12.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 13.1 MB/s eta 0:00:00

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [10]:
from Bio import Entrez
from time import sleep
from datasets import Dataset
import pandas as pd
import os
import re
from transformers import BertTokenizer
import random
from transformers import DataCollatorForLanguageModeling
from transformers import BertForMaskedLM
from transformers import TrainingArguments, Trainer

In [11]:
Entrez.email = "edisonzjy@gmail.com"
search_term = "biomedical research"
max_articles = 10000
batch_size = 100

output_file = "pubmed_abstracts.txt"

def fetch_pubmed_abstracts():
    print(f"Searching PubMed for: {search_term}")
    search_handle = Entrez.esearch(db="pubmed", term=search_term, retmax=max_articles)
    record = Entrez.read(search_handle)
    search_handle.close()

    pmids = record["IdList"]
    print(f"Found {len(pmids)} articles.")

    with open(output_file, "w", encoding="utf-8") as f:
        for start in range(0, len(pmids), batch_size):
            end = min(start + batch_size, len(pmids))
            batch_pmids = pmids[start:end]
            print(f"Fetching batch: {start + 1}–{end}")
            try:
                handle = Entrez.efetch(
                    db="pubmed",
                    id=",".join(batch_pmids),
                    rettype="abstract",
                    retmode="text"
                )
                data = handle.read()
                handle.close()

                articles = []
                raw_entries = data.split("\n\n")
                for entry in raw_entries:
                    match = re.search(r"(?:\n\n)?(?:[A-Z].+?\.){2,}", entry, re.DOTALL)
                    if match:
                        cleaned = match.group().strip().replace("\n", " ")
                        articles.append(cleaned)
                for article in articles:
                    f.write(article + "\n")

                sleep(0.5)
            except Exception as e:
                print(f"Error fetching batch {start + 1}–{end}: {e}")
                sleep(2)

    print(f"\nDone. Saved clean abstracts to: {output_file}")

if __name__ == "__main__":
    fetch_pubmed_abstracts()

Searching PubMed for: biomedical research
Found 9999 articles.
Fetching batch: 1–100
Fetching batch: 101–200
Fetching batch: 201–300
Fetching batch: 301–400
Fetching batch: 401–500
Fetching batch: 501–600
Fetching batch: 601–700
Fetching batch: 701–800
Fetching batch: 801–900
Fetching batch: 901–1000
Fetching batch: 1001–1100
Fetching batch: 1101–1200
Fetching batch: 1201–1300
Fetching batch: 1301–1400
Fetching batch: 1401–1500
Fetching batch: 1501–1600
Fetching batch: 1601–1700
Fetching batch: 1701–1800
Fetching batch: 1801–1900
Fetching batch: 1901–2000
Fetching batch: 2001–2100
Fetching batch: 2101–2200
Fetching batch: 2201–2300
Fetching batch: 2301–2400
Fetching batch: 2401–2500
Fetching batch: 2501–2600
Fetching batch: 2601–2700
Fetching batch: 2701–2800
Fetching batch: 2801–2900
Fetching batch: 2901–3000
Fetching batch: 3001–3100
Fetching batch: 3101–3200
Fetching batch: 3201–3300
Fetching batch: 3301–3400
Fetching batch: 3401–3500
Fetching batch: 3501–3600
Fetching batch: 3601–3

In [20]:
# search_pipeline_clinicalbert.py

import os
import pandas as pd
import numpy as np
import torch
import faiss
from tqdm import tqdm

# --------------------- CONFIG ---------------------
    
MIMIC_FILE = "final_combined_notes.csv"
PUBMED_FILE = "pubmed_abstracts.txt"
TOP_K = 3

# ------------------ LOAD MODEL --------------------
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("pritamdeka/BioBERT-mnli-snli-scinli-scitail-mednli-stsb")

def get_bert_embeddings(texts, batch_size=16):
    return model.encode(texts, batch_size=batch_size, show_progress_bar=True, convert_to_numpy=True)

In [13]:
# ------------------ LOADERS -----------------------
def load_mimic_notes_with_metadata(csv_path, text_column="TEXT"):
    df = pd.read_csv(
        csv_path,
        on_bad_lines='skip',   # Skip malformed rows
        engine='python',       # Use Python engine (slower, more tolerant)
    )
    df = df.dropna(subset=[text_column])
    documents = df[text_column].tolist()
    metadata = df.drop(columns=[text_column]).to_dict(orient='records')
    return documents, metadata

In [14]:
def load_pubmed_documents(file_path):
    documents = []
    metadata = []

    with open(file_path, "r", encoding="utf-8") as f:
        lines = f.readlines()

    for idx, line in enumerate(lines):
        text = line.strip()
        if text:
            documents.append(text)
            metadata.append({"abstract_id": idx + 1})

    return documents, metadata

In [15]:
# ------------------ INDEXING ----------------------

def build_faiss_index(documents, batch_size=16):
    print(f"Embedding {len(documents)} documents...")
    embeddings = get_bert_embeddings(documents, batch_size=batch_size)

    dim = embeddings.shape[1]
    index = faiss.IndexFlatL2(dim)
    index.add(np.array(embeddings))

    return index, embeddings

In [16]:
# ------------------ SEARCH ------------------------

def search_documents(query, index, documents, metadata, top_k=5):
    query_vec = get_bert_embeddings([query])
    distances, indices = index.search(np.array(query_vec), top_k)

    results = []
    for idx in indices[0]:
        results.append({
            "document": documents[idx][:500] + "..." if len(documents[idx]) > 500 else documents[idx],
            "metadata": metadata[idx]
        })
    return results

In [21]:
# ------------------ MAIN --------------------------

def main():
    print("\n--- Loading MIMIC-III Notes ---")
    mimic_docs, mimic_meta = load_mimic_notes_with_metadata(MIMIC_FILE)
    mimic_index, _ = build_faiss_index(mimic_docs)

    print("\n--- Loading PubMed Abstracts ---")
    pubmed_docs, pubmed_meta = load_pubmed_documents(PUBMED_FILE)
    pubmed_index, _ = build_faiss_index(pubmed_docs)

    query = input("\nEnter a medical query: ").strip()

    print("\n--- Top MIMIC-III Notes ---")
    mimic_results = search_documents(query, mimic_index, mimic_docs, mimic_meta, top_k=TOP_K)
    for i, res in enumerate(mimic_results):
        print(f"\nResult {i+1}")
        print("SUBJECT_ID:", res["metadata"].get("SUBJECT_ID"))
        print("HADM_ID:", res["metadata"].get("HADM_ID"))
        print("Drugs Prescribed:", res["metadata"].get("NUM_DRUGS"))
        print("Procedures:", res["metadata"].get("NUM_PROC_ITEMS"))
        print("Note Snippet:", res["document"])

    print("\n--- Top PubMed Abstracts ---")
    pubmed_results = search_documents(query, pubmed_index, pubmed_docs, pubmed_meta, top_k=TOP_K)
    for i, res in enumerate(pubmed_results):
        print(f"\nResult {i+1}")
        print("Abstract ID:", res["metadata"].get("abstract_id"))
        print("Abstract Snippet:", res["document"])


if __name__ == "__main__":
    main()


--- Loading MIMIC-III Notes ---
Embedding 2068 documents...


Batches:   0%|          | 0/130 [00:00<?, ?it/s]


--- Loading PubMed Abstracts ---
Embedding 2429 documents...


Batches:   0%|          | 0/152 [00:00<?, ?it/s]


Enter a medical query:  patient with pneumonia and low oxygen levels



--- Top MIMIC-III Notes ---


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


Result 1
SUBJECT_ID: 20143
HADM_ID: 176761
Drugs Prescribed: nan
Procedures: nan
Note Snippet: Admission Date:  [**2148-10-29**]     Discharge Date:  [**2148-10-29**]


Service:

CHIEF COMPLAINT:  Shortness of breath.

HISTORY OF PRESENT ILLNESS:  The patient is an 89 year-old
female with a history of congestive heart failure and
osteomyelitis of her left femur who presented with shortness
of breath and decreased O2 saturations on room air in the
setting of two days of foul smelling diarrhea.  The patient
lives at [**Hospital3 2558**] and was transferred today to [**Hospital1 1444**] Emer...

Result 2
SUBJECT_ID: 11866
HADM_ID: 148480
Drugs Prescribed: nan
Procedures: nan
Note Snippet: Admission Date:  [**2163-3-7**]       Discharge Date:

Date of Birth:   [**2108-5-28**]       Sex:  M

Service:  Medicine - [**Doctor Last Name **]

CHIEF COMPLAINT:  Hypoxic respiratory failure and hemodynamic
instability.

HISTORY OF PRESENT ILLNESS:  The patient is a 54 year old
white male with a his

Batches:   0%|          | 0/1 [00:00<?, ?it/s]


Result 1
Abstract ID: 1132
Abstract Snippet: BackgroundNeonates are more susceptible to acute respiratory failure than older  children. It is unknown to what extent high-flow nasal cannula (HFNC) alters  intrathoracic pressure (ITP), potentially decreasing cardiac output (CO) due to  cardiopulmonary interactions. This study evaluated the impact of flow titration  on tracheal pressure (a surrogate for ITP) and CO via HFNC in an established  porcine model of neonatal respiratory failure. Unlike prior research, this study  examines both ITP a...

Result 2
Abstract ID: 1929
Abstract Snippet: The primary cause of death from opioid overdose is opioid-induced respiratory  depression (OIRD), characterized by severe suppression of respiratory rate,  destabilized breathing patterns, hypercapnia, and heightened risk of apnea. The  retrotrapezoid nucleus (RTN), a critical chemosensitive brainstem region in the  rostral ventrolateral medullary reticular formation, contains  Phox2b+/neuromedin-B (N

In [ ]:
# kebiolm
# sapbert once
# biolinkbert